In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../00.module')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

df = pd.read_excel('./menu.xlsx',
                   header=0,
                   dtype=str).fillna('')

list_sku = df['Sku'].tolist()
list_url = df['Url'].tolist()

print('总数量：' + str(len(list_url)))
print()

# = = = = = = = = = = = = = = =

df = pd.DataFrame()

# = = = = = = = = = = = = = = =

import requests

from bs4 import BeautifulSoup

from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for sku, url in zip(list_sku, list_url):
    work.put_nowait((sku, url))

def crawler():
    global df
    
    while not work.empty():
        sku, url = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        status = 'error'
        for _ in range(101):
            try:
                resp = requests.get(url,
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                soup = BeautifulSoup(resp, 'lxml')
                
                # = = = = = = = = = = = = = = =
                
                html = etree.HTML(str(soup))

                sku_status = ''.join(html.xpath('//div[@class="product-basic-details__text--part"]/span/text()')).replace(sku.replace('# ', '#'), '').replace('(', '').replace(')', '').strip()

                # = = = = = = = = = = = = = = =
                
                title = html.xpath('//div[@class="product-basic-details__text--title"]/h3/text()')[0].strip()
                
                # = = = = = = = = = = = = = = =
                
                summary = ''.join(html.xpath('//div[@class="product-basic-details__text--summary"]/div/text()')).strip()
                
                # = = = = = = = = = = = = = = =
                
                vehicle = ''.join(html.xpath('//div[@class="product-basic-details__applicable-car"]/text()')).strip().replace('; ', '\n')
                
                # = = = = = = = = = = = = = = =
                
                list_tr_1 = html.xpath('//table[@class="product-basic-details__table"]/tr')

                # = = = = = = = = = = = = = = =
                
                list_tr_2 = html.xpath('//div[@class="product-basic-details__text--section"]/table/tr')
                
                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'status': 'ok',
                                         '序号': '',
                                         'Sku': sku,
                                         'Status': sku_status,
                                         'Title': title,
                                         'Summary': summary,
                                         'Vehicle': vehicle,
                                         'Pic': '',
                                         'Url': url,
                                         'Src': ''}])
                
                for tr in list_tr_1:
                    if tr.xpath('./td/text()')[0].strip() != '':
                        df_temp.loc[0, tr.xpath('./td/text()')[0].strip()[:-1].upper()] = tr.xpath('./td/text()')[1].strip()

                for tr in list_tr_2:
                    df_temp.loc[0, tr.xpath('./td/text()')[0].strip().lower()] = tr.xpath('./td/text()')[1].strip()
                
                status = 'ok'
                
                break
                
            except:
                time.sleep(0.5)
                continue

        if status == 'error':
            df_temp = pd.DataFrame([{'status': 'error',
                                     'Url': url}])

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')

        print(url +  '  <->  [' + status + '] - 剩余数量：' + str(work.qsize()) + ' - 当前时间：' + datetime.now().strftime('%H:%M:%S'))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

df = df.drop_duplicates(ignore_index=True)

print()
print('爬虫输出中')
df.to_excel('./part_' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
print()
print('爬虫结束')

总数量：2477

https://www.centricparts.com/disc-brake-rotor-6853004  <->  [ok] - 剩余数量：2457 - 当前时间：16:00:39
https://www.centricparts.com/disc-brake-rotor-6854102  <->  [ok] - 剩余数量：2456 - 当前时间：16:00:39
https://www.centricparts.com/disc-brake-rotor-121-11003  <->  [ok] - 剩余数量：2455 - 当前时间：16:00:39
https://www.centricparts.com/disc-brake-rotor-121-07000  <->  [ok] - 剩余数量：2454 - 当前时间：16:00:39
https://www.centricparts.com/disc-brake-rotor-121-11006  <->  [ok] - 剩余数量：2453 - 当前时间：16:00:39
https://www.centricparts.com/disc-brake-rotor-121-07006  <->  [ok] - 剩余数量：2452 - 当前时间：16:00:39
https://www.centricparts.com/disc-brake-rotor-121-07001  <->  [ok] - 剩余数量：2451 - 当前时间：16:00:39
https://www.centricparts.com/disc-brake-rotor-121-11002  <->  [ok] - 剩余数量：2450 - 当前时间：16:00:39
https://www.centricparts.com/disc-brake-rotor-121-07003  <->  [ok] - 剩余数量：2449 - 当前时间：16:00:40
https://www.centricparts.com/disc-brake-rotor-121-10009  <->  [ok] - 剩余数量：2448 - 当前时间：16:00:40
https://www.centricparts.com/disc-brake-roto

https://www.centricparts.com/disc-brake-rotor-121-22023  <->  [ok] - 剩余数量：2370 - 当前时间：16:01:01
https://www.centricparts.com/disc-brake-rotor-121-22024  <->  [ok] - 剩余数量：2369 - 当前时间：16:01:01
https://www.centricparts.com/disc-brake-rotor-121-22027  <->  [ok] - 剩余数量：2368 - 当前时间：16:01:01
https://www.centricparts.com/disc-brake-rotor-7356388  <->  [ok] - 剩余数量：2367 - 当前时间：16:01:02
https://www.centricparts.com/disc-brake-rotor-121-04004  <->  [ok] - 剩余数量：2366 - 当前时间：16:01:02
https://www.centricparts.com/disc-brake-rotor-121-22029  <->  [ok] - 剩余数量：2365 - 当前时间：16:01:03
https://www.centricparts.com/disc-brake-rotor-7356389  <->  [ok] - 剩余数量：2364 - 当前时间：16:01:03
https://www.centricparts.com/disc-brake-rotor-7356395  <->  [ok] - 剩余数量：2363 - 当前时间：16:01:03
https://www.centricparts.com/disc-brake-rotor-121-22026  <->  [ok] - 剩余数量：2362 - 当前时间：16:01:04
https://www.centricparts.com/disc-brake-rotor-8791974  <->  [ok] - 剩余数量：2361 - 当前时间：16:01:04
https://www.centricparts.com/disc-brake-rotor-7972249  <->

https://www.centricparts.com/disc-brake-rotor-121-33042  <->  [ok] - 剩余数量：2282 - 当前时间：16:01:28
https://www.centricparts.com/disc-brake-rotor-121-33024  <->  [ok] - 剩余数量：2281 - 当前时间：16:01:29
https://www.centricparts.com/disc-brake-rotor-121-33044  <->  [ok] - 剩余数量：2280 - 当前时间：16:01:29
https://www.centricparts.com/disc-brake-rotor-121-33046  <->  [ok] - 剩余数量：2279 - 当前时间：16:01:29
https://www.centricparts.com/disc-brake-rotor-121-33048  <->  [ok] - 剩余数量：2278 - 当前时间：16:01:30
https://www.centricparts.com/disc-brake-rotor-121-33078  <->  [ok] - 剩余数量：2277 - 当前时间：16:01:31
https://www.centricparts.com/disc-brake-rotor-121-33084  <->  [ok] - 剩余数量：2276 - 当前时间：16:01:31
https://www.centricparts.com/disc-brake-rotor-121-33080  <->  [ok] - 剩余数量：2275 - 当前时间：16:01:31
https://www.centricparts.com/disc-brake-rotor-121-33079  <->  [ok] - 剩余数量：2274 - 当前时间：16:01:31
https://www.centricparts.com/disc-brake-rotor-121-33045  <->  [ok] - 剩余数量：2273 - 当前时间：16:01:31
https://www.centricparts.com/disc-brake-rotor-1057

https://www.centricparts.com/disc-brake-rotor-121-33155  <->  [ok] - 剩余数量：2194 - 当前时间：16:01:56
https://www.centricparts.com/disc-brake-rotor-7356383  <->  [ok] - 剩余数量：2193 - 当前时间：16:01:56
https://www.centricparts.com/disc-brake-rotor-121-33162  <->  [ok] - 剩余数量：2192 - 当前时间：16:01:57
https://www.centricparts.com/disc-brake-rotor-121-33160  <->  [ok] - 剩余数量：2191 - 当前时间：16:01:57
https://www.centricparts.com/disc-brake-rotor-121-33168  <->  [ok] - 剩余数量：2190 - 当前时间：16:01:57
https://www.centricparts.com/disc-brake-rotor-7882296  <->  [ok] - 剩余数量：2189 - 当前时间：16:01:58
https://www.centricparts.com/disc-brake-rotor-121-34077  <->  [ok] - 剩余数量：2188 - 当前时间：16:01:58
https://www.centricparts.com/disc-brake-rotor-121-33164  <->  [ok] - 剩余数量：2187 - 当前时间：16:01:58
https://www.centricparts.com/disc-brake-rotor-7882303  <->  [ok] - 剩余数量：2186 - 当前时间：16:01:58
https://www.centricparts.com/disc-brake-rotor-121-33179  <->  [ok] - 剩余数量：2185 - 当前时间：16:01:59
https://www.centricparts.com/disc-brake-rotor-121-33153 

https://www.centricparts.com/disc-brake-rotor-121-34141  <->  [ok] - 剩余数量：2107 - 当前时间：16:02:22
https://www.centricparts.com/disc-brake-rotor-121-34138  <->  [ok] - 剩余数量：2106 - 当前时间：16:02:22
https://www.centricparts.com/disc-brake-rotor-121-34132  <->  [ok] - 剩余数量：2105 - 当前时间：16:02:22
https://www.centricparts.com/disc-brake-rotor-121-34139  <->  [ok] - 剩余数量：2104 - 当前时间：16:02:22
https://www.centricparts.com/disc-brake-rotor-121-34146  <->  [ok] - 剩余数量：2103 - 当前时间：16:02:23
https://www.centricparts.com/disc-brake-rotor-121-34150  <->  [ok] - 剩余数量：2102 - 当前时间：16:02:23
https://www.centricparts.com/disc-brake-rotor-121-34144  <->  [ok] - 剩余数量：2101 - 当前时间：16:02:23
https://www.centricparts.com/disc-brake-rotor-121-34154  <->  [ok] - 剩余数量：2100 - 当前时间：16:02:23
https://www.centricparts.com/disc-brake-rotor-121-34143  <->  [ok] - 剩余数量：2099 - 当前时间：16:02:23
https://www.centricparts.com/disc-brake-rotor-121-34159  <->  [ok] - 剩余数量：2098 - 当前时间：16:02:23
https://www.centricparts.com/disc-brake-rotor-121-

...

https://www.centricparts.com/disc-brake-rotor-121-67049  <->  [ok] - 剩余数量：354 - 当前时间：16:08:41
https://www.centricparts.com/disc-brake-rotor-121-67036  <->  [ok] - 剩余数量：353 - 当前时间：16:08:41
https://www.centricparts.com/disc-brake-rotor-121-67040  <->  [ok] - 剩余数量：352 - 当前时间：16:08:41
https://www.centricparts.com/disc-brake-rotor-121-67046  <->  [ok] - 剩余数量：351 - 当前时间：16:08:41
https://www.centricparts.com/disc-brake-rotor-121-67039  <->  [ok] - 剩余数量：350 - 当前时间：16:08:41
https://www.centricparts.com/disc-brake-rotor-121-67051  <->  [ok] - 剩余数量：349 - 当前时间：16:08:42
https://www.centricparts.com/disc-brake-rotor-121-67052  <->  [ok] - 剩余数量：348 - 当前时间：16:08:43
https://www.centricparts.com/disc-brake-rotor-121-67054  <->  [ok] - 剩余数量：347 - 当前时间：16:08:43
https://www.centricparts.com/disc-brake-rotor-121-67055  <->  [ok] - 剩余数量：346 - 当前时间：16:08:43
https://www.centricparts.com/disc-brake-rotor-121-67063  <->  [ok] - 剩余数量：345 - 当前时间：16:08:43
https://www.centricparts.com/disc-brake-rotor-121-67059  <->

https://www.centricparts.com/disc-brake-rotor-6854477  <->  [ok] - 剩余数量：265 - 当前时间：16:08:59
https://www.centricparts.com/disc-brake-rotor-121-99041  <->  [ok] - 剩余数量：264 - 当前时间：16:08:59
https://www.centricparts.com/disc-brake-rotor-121-99042  <->  [ok] - 剩余数量：263 - 当前时间：16:09:00
https://www.centricparts.com/disc-brake-rotor-121-99040  <->  [ok] - 剩余数量：262 - 当前时间：16:09:00
https://www.centricparts.com/disc-brake-rotor-121-99043  <->  [ok] - 剩余数量：261 - 当前时间：16:09:00
https://www.centricparts.com/disc-brake-rotor-121-99045  <->  [ok] - 剩余数量：260 - 当前时间：16:09:01
https://www.centricparts.com/disc-brake-rotor-121-99048  <->  [ok] - 剩余数量：259 - 当前时间：16:09:01
https://www.centricparts.com/disc-brake-rotor-121-99052  <->  [ok] - 剩余数量：258 - 当前时间：16:09:01
https://www.centricparts.com/disc-brake-rotor-121-99053  <->  [ok] - 剩余数量：257 - 当前时间：16:09:01
https://www.centricparts.com/disc-brake-rotor-121-99050  <->  [ok] - 剩余数量：256 - 当前时间：16:09:01
https://www.centricparts.com/disc-brake-rotor-121-99044  <->  

https://www.centricparts.com/disc-brake-rotor-8792033  <->  [ok] - 剩余数量：177 - 当前时间：16:09:19
https://www.centricparts.com/disc-brake-rotor-121-66006  <->  [ok] - 剩余数量：176 - 当前时间：16:09:20
https://www.centricparts.com/disc-brake-rotor-121-66007  <->  [ok] - 剩余数量：175 - 当前时间：16:09:20
https://www.centricparts.com/disc-brake-rotor-121-99083  <->  [ok] - 剩余数量：174 - 当前时间：16:09:20
https://www.centricparts.com/disc-brake-rotor-121-66008  <->  [ok] - 剩余数量：173 - 当前时间：16:09:20
https://www.centricparts.com/disc-brake-rotor-121-66005  <->  [ok] - 剩余数量：172 - 当前时间：16:09:21
https://www.centricparts.com/disc-brake-rotor-121-99084  <->  [ok] - 剩余数量：171 - 当前时间：16:09:21
https://www.centricparts.com/disc-brake-rotor-121-99086  <->  [ok] - 剩余数量：170 - 当前时间：16:09:21
https://www.centricparts.com/disc-brake-rotor-121-99085  <->  [ok] - 剩余数量：169 - 当前时间：16:09:21
https://www.centricparts.com/disc-brake-rotor-121-99087  <->  [ok] - 剩余数量：168 - 当前时间：16:09:21
https://www.centricparts.com/disc-brake-rotor-121-99082  <->  

https://www.centricparts.com/disc-brake-rotor-121-63048  <->  [ok] - 剩余数量：88 - 当前时间：16:09:37
https://www.centricparts.com/disc-brake-rotor-121-65067  <->  [ok] - 剩余数量：87 - 当前时间：16:09:37
https://www.centricparts.com/disc-brake-rotor-121-65072  <->  [ok] - 剩余数量：86 - 当前时间：16:09:37
https://www.centricparts.com/disc-brake-rotor-121-63047  <->  [ok] - 剩余数量：85 - 当前时间：16:09:37
https://www.centricparts.com/disc-brake-rotor-121-65065  <->  [ok] - 剩余数量：84 - 当前时间：16:09:38
https://www.centricparts.com/disc-brake-rotor-121-63049  <->  [ok] - 剩余数量：83 - 当前时间：16:09:38
https://www.centricparts.com/disc-brake-rotor-121-63051  <->  [ok] - 剩余数量：82 - 当前时间：16:09:39
https://www.centricparts.com/disc-brake-rotor-121-63050  <->  [ok] - 剩余数量：81 - 当前时间：16:09:39
https://www.centricparts.com/disc-brake-rotor-121-63052  <->  [ok] - 剩余数量：80 - 当前时间：16:09:39
https://www.centricparts.com/disc-brake-rotor-121-63054  <->  [ok] - 剩余数量：79 - 当前时间：16:09:39
https://www.centricparts.com/disc-brake-rotor-121-65068  <->  [ok] - 剩

https://www.centricparts.com/disc-brake-rotor-121-85001  <->  [ok] - 剩余数量：0 - 当前时间：16:09:57
https://www.centricparts.com/disc-brake-rotor-121-84001  <->  [ok] - 剩余数量：0 - 当前时间：16:09:57
https://www.centricparts.com/disc-brake-rotor-6853402  <->  [ok] - 剩余数量：0 - 当前时间：16:09:57
https://www.centricparts.com/disc-brake-rotor-121-84003  <->  [ok] - 剩余数量：0 - 当前时间：16:09:58
https://www.centricparts.com/disc-brake-rotor-121-86003  <->  [ok] - 剩余数量：0 - 当前时间：16:09:58
https://www.centricparts.com/disc-brake-rotor-121-86009  <->  [ok] - 剩余数量：0 - 当前时间：16:09:58
https://www.centricparts.com/disc-brake-rotor-6853403  <->  [ok] - 剩余数量：0 - 当前时间：16:09:59
https://www.centricparts.com/disc-brake-rotor-121-86016  <->  [ok] - 剩余数量：0 - 当前时间：16:09:59
https://www.centricparts.com/disc-brake-rotor-121-86006  <->  [ok] - 剩余数量：0 - 当前时间：16:09:59
https://www.centricparts.com/disc-brake-rotor-6854629  <->  [ok] - 剩余数量：0 - 当前时间：16:09:59
https://www.centricparts.com/disc-brake-rotor-6853248  <->  [ok] - 剩余数量：0 - 当前时间：16:09